In [41]:
import os
import pandas as pd
import geopandas as gpd
from scipy.spatial import KDTree
from sklearn.preprocessing import StandardScaler

In [22]:
main_dir = os.getcwd().rsplit("\\", 2)[0]

In [47]:
# first read all the cleaned data files
weather_df = pd.read_csv(f"{main_dir}/data/processed/cleaned_weather_data.csv")
energy_df = pd.read_csv(f"{main_dir}/data/processed/final_cleaned_energy_hr.csv")
annual_energy_df = pd.read_csv(f"{main_dir}/data/processed/final_cleaned_energy_year.csv")
usgs_df = pd.read_csv(f"{main_dir}/data/processed/final_cleaned_usgs.csv")
osm_raw_df =   pd.read_csv(f"{main_dir}/data/processed/osm_lat_long_all_processed.csv")
osm_processed_df = pd.read_csv(f"{main_dir}/data/processed/osm_site_processed.csv")
sentiment_df = pd.read_csv(f"{main_dir}/data/processed/combined_sentiment_data.csv")
annual_energy_df.rename(columns={
    'generation': 'capacity_MW'
}, inplace=True)

In [48]:
usgs_df.head()

,latitude,longitude,capacity_MW,plantcode
0,32.19000,-101.43630,34.32,54979
1,29.18049,-100.18203,99.00,58000
2,32.87516,-100.58740,250.12,62142
3,32.87976,-100.60146,250.12,62142
4,32.89460,-100.60215,250.12,62142


In [49]:
# Merge USGS with annual energy data to get energy source 
usgs_df = pd.merge(usgs_df, annual_energy_df[['plantcode', 'energy_source','plantname']],
                   on='plantcode', how='left')

In [50]:
usgs_df['energy_source'] = usgs_df['energy_source'].fillna('wind')

In [51]:
# drop rows with missing coordinates
usgs_df = usgs_df.dropna(subset=['latitude', 'longitude'])

In [52]:
usgs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393409 entries, 0 to 393408
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   latitude       393409 non-null  float64
 1   longitude      393409 non-null  float64
 2   capacity_MW    393409 non-null  float64
 3   plantcode      393409 non-null  int64  
 4   energy_source  393409 non-null  object 
 5   plantname      393069 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 18.0+ MB


In [53]:

## Create coordinate an capacity mappings for wind and solar
wind_coords = usgs_df[usgs_df['energy_source'] == 'wind'][['latitude', 'longitude', 'capacity_MW']].mean().to_dict()


In [54]:
solar_coords = osm_raw_df[osm_raw_df['energy_source'] == 'solar'][['latitude', 'longitude','capacity_MW']].mean().to_dict()

In [55]:
solar_capacity = annual_energy_df[annual_energy_df['energy_source'] == 'solar']['capacity_MW'].mean()

In [56]:
# Assign coordinates and capacity to hourly energy data
energy_df['latitude'] = energy_df['energy_source'].map({
    'wind': wind_coords.get('latitude'),
    'solar': solar_coords.get('latitude',31.9686)
})
energy_df['longitude'] = energy_df['energy_source'].map({
    'wind': wind_coords.get('longitude'),
    'solar': solar_coords.get('longitude',-99.9018)
})
energy_df['capacity_MW'] = energy_df['energy_source'].map({
    'wind': wind_coords.get('capacity_MW'),
    'solar': solar_capacity if not pd.isna(solar_capacity) else osm_raw_df['capacity_MW'].mean()
})

In [57]:
energy_df.head()

,date,energy_source,power_MW,maintenance_status,respondent,latitude,longitude,capacity_MW
0,2025-07-08 04:00:00,solar,0.0,1,TEX,31.140744,-98.229480,162956.921267
1,2025-07-08 04:00:00,wind,11946.0,0,TEX,31.987395,-100.410991,192.115738
2,2025-07-08 03:00:00,solar,0.0,1,TEX,31.140744,-98.229480,162956.921267
3,2025-07-08 03:00:00,wind,10473.0,0,TEX,31.987395,-100.410991,192.115738
4,2025-07-08 02:00:00,solar,941.0,0,TEX,31.140744,-98.229480,162956.921267


In [58]:
annual_energy_df.head()

,year,plantcode,plantname,energy_source,capacity_MW
0,2024,62562,"High Lonesome Wind Power, LLC Hybrid",wind,1184601.84
1,2024,56291,Horse Hollow Wind Energy Center,wind,2617502.00
2,2024,56291,Horse Hollow Wind Energy Center,wind,2617502.00
3,2024,56395,Mesquite Wind Power LLC,wind,388599.00
4,2024,56395,Mesquite Wind Power LLC,wind,388599.00


In [59]:
# merge with annal energy data for plant names
# 2025-07-08 04:00:00	- get year from date
energy_df['date'] = pd.to_datetime(energy_df['date'])
energy_df['year'] = energy_df['date'].dt.year
energy_df = pd.merge(energy_df, annual_energy_df[['year','plantcode', 'plantname']],
                     on='year', how='left')

MemoryError: Unable to allocate 1.88 GiB for an array with shape (4, 63204202) and data type float64

In [ ]:
energy_df.head()

,date,energy_source,power_MW,maintenance_status,respondent,latitude,longitude,capacity_MW,year,plantcode,plantname
0,2025-07-08 04:00:00,solar,0.0,1,TEX,31.140744,-98.229480,162956.921267,2025,NaN,NaN
1,2025-07-08 04:00:00,wind,11946.0,0,TEX,31.987395,-100.410991,192.115738,2025,NaN,NaN
2,2025-07-08 03:00:00,solar,0.0,1,TEX,31.140744,-98.229480,162956.921267,2025,NaN,NaN
3,2025-07-08 03:00:00,wind,10473.0,0,TEX,31.987395,-100.410991,192.115738,2025,NaN,NaN
4,2025-07-08 02:00:00,solar,941.0,0,TEX,31.140744,-98.229480,162956.921267,2025,NaN,NaN


In [ ]:
# count nan values in each column
energy_df['plantname'].isna().sum()

np.int64(9034)

## Preprocessing for  Cleaned weather data

In [ ]:

weather_df.head()

,date,station,wind_speed,precipitation,temperature_avg,wind_volatility
0,2015-01-01,GHCND:USC00412114,0.4,9.9,4.15,NaN
1,2015-01-01,GHCND:USW00053903,2.0,31.8,4.75,NaN
2,2015-01-01,GHCND:USW00053902,2.4,11.9,5.55,NaN
3,2015-01-01,GHCND:USW00023091,2.1,0.8,-4.90,NaN
4,2015-01-01,GHCND:USW00023047,2.6,0.0,-5.45,NaN


In [ ]:
### preprocessing for weather data
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459640 entries, 0 to 459639
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             459640 non-null  object 
 1   station          459640 non-null  object 
 2   wind_speed       459640 non-null  float64
 3   precipitation    459640 non-null  float64
 4   temperature_avg  459640 non-null  float64
 5   wind_volatility  459473 non-null  float64
dtypes: float64(4), object(2)
memory usage: 21.0+ MB


In [40]:
weather_df['date'] = pd.to_datetime(weather_df['date'])

In [60]:
# add header for this 
#ID            1-11   Character
# LATITUDE     13-20   Real
# LONGITUDE    22-30   Real
# ELEVATION    32-37   Real
# STATE        39-40   Character
# NAME         42-71   Character
# GSN FLAG     73-75   Character
# HCN/CRN FLAG 77-79   Character
# WMO ID 

# ACW00011604	17.1167	-61.7833	10.1	ST JOHNS COOLIDGE FLD	Unnamed: 5	Unnamed: 6	Unnamed: 7


station_cords = pd.read_fwf(
    "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt")
station_cords.columns = [
    'station_id', 'latitude', 'longitude', 'elevation', 'name', 
    'state', 'gsn_flag', 'wmo_id'
]
station_cords.head()

,station_id,latitude,longitude,elevation,name,state,gsn_flag,wmo_id
0,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,NaN
1,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP,NaN,GSN,41196.0
2,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL,NaN,NaN,41194.0
3,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL,NaN,NaN,41217.0
4,AEM00041218,24.2620,55.6090,264.9,AL AIN INTL,NaN,NaN,41218.0


In [61]:
station_cords = station_cords[station_cords['station_id'].str.startswith(('USC', 'USW'))]

In [62]:
station_cords.head()

,station_id,latitude,longitude,elevation,name,state,gsn_flag,wmo_id
101746,USC00010008,31.5703,-85.2483,139.0,ABBEVILLE,NaN,NaN,NaN
101747,USC00010063,34.2108,-87.1783,239.6,ADDISON,NaN,NaN,NaN
101748,USC00010071,34.4167,-87.3167,302.1,ADDISON CNTRL TWR,NaN,NaN,NaN
101749,USC00010117,34.6833,-86.8833,182.9,ALABAMA STATE FARM,NaN,NaN,NaN
101750,USC00010125,31.1333,-85.0667,34.1,ALAGA,NaN,NaN,NaN


In [63]:
weather_df_copy = weather_df.copy()

In [64]:
weather_df_copy.head()

,date,station,wind_speed,precipitation,temperature_avg,wind_volatility
0,2015-01-01,GHCND:USC00412114,0.4,9.9,4.15,NaN
1,2015-01-01,GHCND:USW00053903,2.0,31.8,4.75,NaN
2,2015-01-01,GHCND:USW00053902,2.4,11.9,5.55,NaN
3,2015-01-01,GHCND:USW00023091,2.1,0.8,-4.90,NaN
4,2015-01-01,GHCND:USW00023047,2.6,0.0,-5.45,NaN


In [65]:
#GHCND:USC00412114	
weather_df_copy['station'] = weather_df_copy['station'].str.replace('GHCND:', '', regex=False)
weather_df_copy.head()

,date,station,wind_speed,precipitation,temperature_avg,wind_volatility
0,2015-01-01,USC00412114,0.4,9.9,4.15,NaN
1,2015-01-01,USW00053903,2.0,31.8,4.75,NaN
2,2015-01-01,USW00053902,2.4,11.9,5.55,NaN
3,2015-01-01,USW00023091,2.1,0.8,-4.90,NaN
4,2015-01-01,USW00023047,2.6,0.0,-5.45,NaN


In [66]:
weather_df_copy = pd.merge(weather_df_copy, station_cords[['station_id', 'latitude', 'longitude']],
                          left_on='station', right_on='station_id', how='left')

In [67]:
weather_df_copy.drop(columns=['station_id'], inplace=True)

In [68]:
weather_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459640 entries, 0 to 459639
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             459640 non-null  object 
 1   station          459640 non-null  object 
 2   wind_speed       459640 non-null  float64
 3   precipitation    459640 non-null  float64
 4   temperature_avg  459640 non-null  float64
 5   wind_volatility  459473 non-null  float64
 6   latitude         459640 non-null  float64
 7   longitude        459640 non-null  float64
dtypes: float64(6), object(2)
memory usage: 28.1+ MB


### Weather dataframe cleaned up

In [69]:
# merge with weather data
gdf_weather = gpd.GeoDataFrame(weather_df_copy,
                               geometry= gpd.points_from_xy(weather_df_copy['longitude'], weather_df_copy['latitude']))

In [70]:
gdf_weather.head()

,date,station,wind_speed,precipitation,temperature_avg,wind_volatility,latitude,longitude,geometry
0,2015-01-01,USC00412114,0.4,9.9,4.15,NaN,31.3072,-95.4508,POINT (-95.4508 31.3072)
1,2015-01-01,USW00053903,2.0,31.8,4.75,NaN,30.7444,-95.5867,POINT (-95.5867 30.7444)
2,2015-01-01,USW00053902,2.4,11.9,5.55,NaN,30.3611,-95.4175,POINT (-95.4175 30.3611)
3,2015-01-01,USW00023091,2.1,0.8,-4.90,NaN,30.9119,102.9172,POINT (102.9172 30.9119)
4,2015-01-01,USW00023047,2.6,0.0,-5.45,NaN,35.2333,101.7092,POINT (101.7092 35.2333)


In [71]:
gdf_energy = gpd.GeoDataFrame(energy_df,
                               geometry=gpd.points_from_xy(energy_df['longitude'], energy_df['latitude']))

In [72]:
gdf_energy.head()

,date,energy_source,power_MW,maintenance_status,respondent,latitude,longitude,capacity_MW,year,geometry
0,2025-07-08 04:00:00,solar,0.0,1,TEX,31.140744,-98.229480,162956.921267,2025,POINT (-98.22948 31.14074)
1,2025-07-08 04:00:00,wind,11946.0,0,TEX,31.987395,-100.410991,192.115738,2025,POINT (-100.41099 31.9874)
2,2025-07-08 03:00:00,solar,0.0,1,TEX,31.140744,-98.229480,162956.921267,2025,POINT (-98.22948 31.14074)
3,2025-07-08 03:00:00,wind,10473.0,0,TEX,31.987395,-100.410991,192.115738,2025,POINT (-100.41099 31.9874)
4,2025-07-08 02:00:00,solar,941.0,0,TEX,31.140744,-98.229480,162956.921267,2025,POINT (-98.22948 31.14074)


In [73]:
print(energy_df.value_counts())
print(gdf_weather.value_counts())

date                 energy_source  power_MW  maintenance_status  respondent  latitude   longitude    capacity_MW    year
2025-07-08 04:00:00  wind           11946.0   0                   TEX         31.987395  -100.410991  192.115738     2025    1
2019-01-01 00:00:00  solar          161.0     0                   TEX         31.140744  -98.229480   162956.921267  2019    1
                     wind           5020.0    0                   TEX         31.987395  -100.410991  192.115738     2019    1
2019-01-01 01:00:00  solar          0.0       1                   TEX         31.140744  -98.229480   162956.921267  2019    1
                     wind           5597.0    0                   TEX         31.987395  -100.410991  192.115738     2019    1
                                                                                                                            ..
2019-01-01 06:00:00  wind           13676.0   0                   TEX         31.987395  -100.410991  192.115738    

In [79]:
import numpy as np
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat/2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

In [81]:
# Extract unique energy locations
unique_energy = energy_df[['latitude', 'longitude']].drop_duplicates()

In [82]:
weather_df_copy.columns

Index(['date', 'station', 'wind_speed', 'precipitation', 'temperature_avg',
       'wind_volatility', 'latitude', 'longitude'],
      dtype='object')

In [83]:
# Build KDTree on weather locations
weather_locs = weather_df_copy[['latitude', 'longitude', 'station']].dropna().to_numpy()

In [89]:
if len(weather_locs) == 0:
    print("Warning: No valid weather station coordinates. Using fallback.")
    energy_df['nearest_station'] = 'unknown'
else:
    tree = KDTree(weather_locs[:, :2])
    dist, idx = tree.query(unique_energy[['latitude', 'longitude']].to_numpy())
    unique_energy['nearest_station'] = weather_locs[idx, 2]
    unique_energy['distance_km'] = dist
    unique_energy = unique_energy[unique_energy['distance_km'] < 100]  # Max 100 km

In [ ]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459640 entries, 0 to 459639
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             459640 non-null  object 
 1   nearest_station  459640 non-null  object 
 2   wind_speed       459640 non-null  float64
 3   precipitation    459640 non-null  float64
 4   temperature_avg  459640 non-null  float64
 5   wind_volatility  459473 non-null  float64
 6   latitude         459640 non-null  float64
 7   longitude        459640 non-null  float64
dtypes: float64(6), object(2)
memory usage: 28.1+ MB


In [93]:
# Merge back to energy_df
energy_df = energy_df.merge(unique_energy[['latitude', 'longitude', 'nearest_station']], 
                               on=['latitude', 'longitude'], how='left')

In [94]:
energy_df.columns

Index(['date', 'energy_source', 'power_MW', 'maintenance_status', 'respondent',
       'latitude', 'longitude', 'capacity_MW', 'year', 'nearest_station'],
      dtype='object')

In [95]:
weather_df_copy = weather_df_copy.rename(columns={'station': 'nearest_station'})

In [105]:
energy_df['date'] = pd.to_datetime(energy_df['date'])
weather_df_copy['date'] = pd.to_datetime(weather_df_copy['date'])

In [106]:
merged_df = energy_df.merge(
        weather_df_copy[['date', 'nearest_station', 'wind_speed', 'precipitation', 'temperature_avg', 'wind_volatility']],
        on=['date', 'nearest_station'],
        how='left'
    )

In [108]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114250 entries, 0 to 114249
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date                114250 non-null  datetime64[ns]
 1   energy_source       114250 non-null  object        
 2   power_MW            114250 non-null  float64       
 3   maintenance_status  114250 non-null  int64         
 4   respondent          114250 non-null  object        
 5   latitude            114250 non-null  float64       
 6   longitude           114250 non-null  float64       
 7   capacity_MW         114250 non-null  float64       
 8   year                114250 non-null  int32         
 9   nearest_station     114250 non-null  object        
 10  wind_speed          3394 non-null    float64       
 11  precipitation       3394 non-null    float64       
 12  temperature_avg     3394 non-null    float64       
 13  wind_volatility     3394 non-

In [109]:
# Merge with sentiment data
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date
sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.date
merged_df = pd.merge(merged_df, sentiment_df, on='date', how='left')
merged_df['sentiment_score'] = merged_df['sentiment_score'].fillna(0)

In [110]:
# Merge with OSM processed data for site_density
merged_df = pd.merge(merged_df, osm_processed_df[['latitude', 'longitude', 'site_density']], 
                     on=['latitude', 'longitude'], how='left')
merged_df['site_density'] = merged_df['site_density'].fillna(0)

In [111]:
# Feature engineering
merged_df['output_efficiency'] = merged_df['power_MW'] / merged_df['capacity_MW'].replace(0, 1)

In [112]:
# Outlier removal
Q1 = merged_df['power_MW'].quantile(0.25)
Q3 = merged_df['power_MW'].quantile(0.75)
IQR = Q3 - Q1
merged_df = merged_df[(merged_df['power_MW'] >= Q1 - 1.5*IQR) & (merged_df['power_MW'] <= Q3 + 1.5*IQR)]

In [113]:
# Scale numerical features
scaler = StandardScaler()
numerical_cols = ['wind_speed', 'precipitation', 'temperature_avg', 'wind_volatility', 'power_MW', 'site_density', 'output_efficiency']
merged_df[numerical_cols] = scaler.fit_transform(merged_df[numerical_cols].fillna(0))

In [116]:
merged_df.head()

,date,energy_source,power_MW,maintenance_status,respondent,latitude,longitude,capacity_MW,year,nearest_station,wind_speed,precipitation,temperature_avg,wind_volatility,sentiment_score,site_density,output_efficiency
0,2025-07-08,solar,-1.015388,1,TEX,31.140744,-98.229480,162956.921267,2025,USW00000229,-0.159343,-0.038485,-0.158559,-0.163295,0.0,0.0,-0.792150
1,2025-07-08,wind,0.688349,0,TEX,31.987395,-100.410991,192.115738,2025,USW00013962,-0.159343,-0.038485,-0.158559,-0.163295,0.0,0.0,0.869441
2,2025-07-08,solar,-1.015388,1,TEX,31.140744,-98.229480,162956.921267,2025,USW00000229,-0.159343,-0.038485,-0.158559,-0.163295,0.0,0.0,-0.792150
3,2025-07-08,wind,0.478270,0,TEX,31.987395,-100.410991,192.115738,2025,USW00013962,-0.159343,-0.038485,-0.158559,-0.163295,0.0,0.0,0.664558
4,2025-07-08,solar,-0.881183,0,TEX,31.140744,-98.229480,162956.921267,2025,USW00000229,-0.159343,-0.038485,-0.158559,-0.163295,0.0,0.0,-0.791995


In [115]:
cleaned_df = merged_df.copy()
cleaned_df.to_csv(f"{main_dir}/data/processed/cleaned_data_final.csv", index=False)